In [1]:
#define initial conditions
include("Markov_Chain_tools.jl")

util= (x, σ)-> x>0 ? (x^(1.0-σ)-1.0)/(1.0-σ) : -Inf
inv_mutil= (x, σ)-> x>0 ? x^(-1.0/σ) : NaN

# 2) Define some parameters
α    = 0.33    # Capital elasticity in production function
β    = 0.96    # Discount factor
δ    = 0.06    # Depreciation rate
σ    = 2.00    # utility curvature (Intertemporal eleasticity of substitution)
param = [α,β,δ,σ]

# 3) Define the shocks -> log(a_t) = ρ \log(a_{t-1}) + (1-ρ)μ + ϵ_t  ϵ_t ~> N(0,σ²)  
# and approximate it by a Markov Chain
ρ    = 0.8     # persistence of the shock
ς    = 0.02    # Volatility of the innovation
μ    = 0       # Average of the process
Nz   = 5       # Number of points in the grid of shocks
la,P = rouwenhorst(Nz,ρ,ς,μ) # If Rouwenhorst's (1995) method is chosen
a    = exp.(la);

# 4) Define the grid
kss  = (α*β/(1.0-β*(1.0-δ)))^(1.0/(1.0-α)) # Deterministic steady state of capital
Δk   = 0.5
kmin = (1.0-Δk)*kss
kmax = (1.0+Δk)*kss
Nk   = 1000
kgrid= range(kmin,stop=kmax,length=Nk)

# 5) define starting values 

Rgrid= a'.*kgrid.^α .+ (1-δ)*kgrid
v0      = util.(Rgrid,σ)/(1.0-β)

1000×5 Matrix{Float64}:
 18.8326  18.9004  18.969   19.0382  19.1082
 18.8422  18.9099  18.9782  19.0473  19.1171
 18.8518  18.9193  18.9875  19.0564  19.126
 18.8614  18.9287  18.9967  19.0655  19.1349
 18.8709  18.938   19.0059  19.0745  19.1438
 18.8804  18.9474  19.0151  19.0835  19.1526
 18.8899  18.9567  19.0242  19.0925  19.1615
 18.8993  18.9659  19.0333  19.1014  19.1702
 18.9087  18.9752  19.0424  19.1103  19.179
 18.9181  18.9844  19.0515  19.1192  19.1877
  ⋮                                  
 22.5091  22.525   22.5413  22.5578  22.5748
 22.5105  22.5264  22.5427  22.5593  22.5762
 22.512   22.5279  22.5441  22.5607  22.5776
 22.5134  22.5293  22.5455  22.5621  22.5789
 22.5149  22.5308  22.547   22.5635  22.5803
 22.5163  22.5322  22.5484  22.5649  22.5817
 22.5178  22.5336  22.5498  22.5663  22.5831
 22.5192  22.5351  22.5512  22.5677  22.5845
 22.5207  22.5365  22.5526  22.5691  22.5859

In [2]:
#From last to first steps. 1 derivative at different points of the grid. Here we assume that the function is differentiable, has no kinks etc. 

function deriv(v,k,i)

        λ= (k[i+1]-k[i])/(k[i+1]- k[i-1])
        dv1= ((v[i+1]-v[i]))/(k[i+1]-k[i])
        dv2= ((v[i]-v[i-1]))/(k[i]- k[i-1])
        dv= (1-λ)*dv1 +λ*dv2

end 

#we need to flag which elements in the corner we don't weight; but we can do it all in the same function, no? 

function apply_deriv(v, k)

    Nk, Nz = size(v)
    dv = similar(v)

    for iz in 1:Nz
        V=v[:,iz]
        dv[1,iz]    =(V[2]-V[1])/(k[2]-k[1])
        dv[Nk,iz]   =(V[Nk]-V[Nk-1])/(k[Nk]-k[Nk-1])
        
        for ik in 2:(Nk-1)
            dv[ik, iz]= deriv(V,k,ik) #in between we just take a convex combination (pending to check)
        end 
    end 
    return dv
end

apply_deriv (generic function with 1 method)

In [3]:
#now we want to recover Kt* from ct*, we will use the tayloy approximation rule Fabrice also used to solve his model. 


function get_capital(R, param,kgrid,zgrid, error)

    α,β,δ,σ = param
    Nz = similar(R) #we want to consider all the shocks 
    Gk        = similar(R)
    k_endo    = similar(R)
    for (iz,z) in enumerate(zgrid)
        k0= kgrid
        crit=1.0
        while crit>error
            F= z*k0.^α .+ (1.0-δ)*k0 -R[:, iz]
            dF= z*α*k0.^(α-1.0).+(1.0-δ)
            k= k0 -(F./dF)
            crit= norm(k-k0, Inf)
            k0= copy(k)
        end 

        k_endo[:, iz]= k0
        kint      = LinearInterpolation(k0,kgrid;extrapolation_bc=Line())
        Gk[:, iz]= kint.(kgrid)
        #here we find the optimal k* given kt+1 and at+1, but we want to invert this to get the policy function. 

    end 
    
    return k_endo,Gk 

end 

#within this loop over iz I can also interpolate, but don't understand clearly what interpolating means in this case. 


get_capital (generic function with 1 method)

In [4]:
#Simulation of the value function and interpolation. Once we have the policy function we have to find the fixed point of the value function; using the usual algorithm. 

function expectation(x,P)
    x*P'
end 




function v_simul(v,param,kgrid,zgrid,P)

        dv   = apply_deriv(v, kgrid)
        Ev    = expectation(v,P)
        Edv   = expectation(dv,P)
        c     = inv_mutil.(β*Edv,σ)
        R     = c.+kgrid

        vn= util.(c, σ) +β*Ev
        k_endo, Gk= get_capital(R, param,kgrid,zgrid, 1e-08)
        Tv=reduce(hcat,[LinearInterpolation(k_endo[:,iz],vn[:,iz];
    extrapolation_bc=Line()).(kgrid) for iz in 1:Nz])
        

    return Tv,Gk

end 

v_simul (generic function with 1 method)

In [5]:
##do the bellman loop 

function bell_eq(v, param, kgrid, zgrid, P)
    α,β,δ,σ   = param
    Nk,Nz     = size(v)    

    crit= 1
    tol= 1e-08
    iter= 0
    local Tv,Gk
    while crit>tol 
        Tv, Gk = v_simul(v, param, kgrid, zgrid, P)
        crit= norm(Tv-v, Inf)
        v=copy(Tv) #don't occupy as much memory. 
        iter +=1
    end 
    println("Iteration: $iter, Crit: $crit")
    return Tv, Gk
end

bell_eq (generic function with 1 method)

In [7]:
v, a= v_simul(v0, param, kgrid, a, P)


([18.373760252714437 18.47969582081177 … 18.691650001400905 18.79756607282159; 18.38228785484987 18.48801163517299 … 18.69954412658048 18.80525362076957; … ; 21.969640323675385 22.00014225406804 … 22.062194916645986 22.093728052975163; 21.97121140547539 22.00168786410108 … 22.063689132395403 22.09519653504081], [3.1924974619373634 3.2245215792331154 … 3.2919011572205 3.3273318680738977; 3.197668585184441 3.2297146941738113 … 3.2971404225786265 3.3325954585955633; … ; 8.14424298284933 8.1907194663893 … 8.288548662663892 8.340012727808048; 8.149091118939637 8.19557804648199 … 8.29342923467323 8.344904888107212])

In [25]:
#apply the function

@time v,Gk = bell_eq(v0,param,kgrid,a,P)
Gc         = Rgrid-Gk;
Gi         = Gk.-(1-δ)*kgrid;

Iteration: 438, Crit: 9.964601588308142e-9
  0.406691 seconds (374.10 k allocations: 1.065 GiB, 6.53% gc time)


In [28]:
using PyPlot

function fsubplot(r,c,i)
    bg=[0.95,0.95,0.95]
    ax=subplot(r,c,i,facecolor=bg)
    for sp in ("bottom","top","left","right")
          ax.spines[sp].set_color("none")
          ax.tick_params(axis="y", width=0, length=0)
          ax.tick_params(axis="x", width=0, length=0)        
    end
    ax.grid(color="w",linestyle="-",alpha=0.5,linewidth=1.5,zorder=0)
    return ax
end

fsubplot (generic function with 1 method)

In [ ]:

using PyPlot
figure(figsize=(12,8))
ax=fsubplot(2,1,1)
ax.plot(kgrid,Gk,color="k",linewidth=1)                               # Plots k(t+1)=G(k(t))
line45, = ax.plot(kgrid,kgrid,color="k",linewidth=0.5,linestyle="--") # Plots the 45° line
line45.set_dashes((12,4))                                          # Change the aspect of the dashes
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
title(L"k_{t+1}=G_k(k_t,a_t)",fontsize=14)                              # put a label on y-axis the L forces LaTeX code

ax=fsubplot(2,2,3)
ax.plot(kgrid,Gc,color="k",linewidth=1)                               # Plots k(t+1)=G(k(t))
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
title(L"c_{t}=G_c(k_t,a_t)",fontsize=14)                              # put a label on y-axis the L forces LaTeX code

ax=fsubplot(2,2,4)
ax.plot(kgrid,Gi,color="k",linewidth=1)                               # Plots k(t+1)=G(k(t))
xlabel(L"k_t",fontsize=12)                                         # put a label on x-axis the L forces LaTeX code
title(L"i_{t}=G_i(k_t,a_t)",fontsize=14)                              # put a label on y-axis the L forces LaTeX code

tight_layout();